In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, XGBRegressor

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score

%pylab inline
data=None
def absolute_correlations(col, df=data):
    #absolute_values = np.abs(df[col])
    corrs = pd.DataFrame(df.select_dtypes(include=[np.number]).corrwith(df[col]), columns=['correlation'])
    corrs['absol'] = np.abs(corrs['correlation'])
    return corrs.sort_values('absol', ascending=False).drop('absol', axis=1).tail(len(corrs)-1)

def numeric(self):
    return self.select_dtypes(include=[np.number])
pd.DataFrame.numeric = numeric

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_json('train.json')

In [3]:
data.head()

,correct_answer,distractor1,distractor2,distractor3,question,support
0,mesophilic organisms,protozoa,gymnosperms,viruses,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ..."
1,coriolis effect,muon effect,centrifugal effect,tropical effect,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...
2,exothermic,unbalanced,reactive,endothermic,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...
3,alpha decay,beta decay,gamma decay,zeta decay,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...
4,smoke and ash,greenhouse gases,carbon and smog,magma,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...


In [7]:
test = pd.read_json('test.json')

In [4]:
data.shape

(11679, 6)

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()


for i in range(len(data)):
    # Identify input_text and target_text
    input_texts = data['question'].str.replace('?', '').values.tolist()
    target_texts = '\t' + data['correct_answer'] + '\n'
    target_texts = target_texts.values.tolist()

In [6]:
input_words = []
target_words = []

for text in input_texts:
    for word in text.split():
        input_words.append(word.lower())
        
for text in target_texts:
    for word in text.split():
        target_words.append(word.lower())

In [7]:
target_words = set(target_words)
input_words = set(input_words)

all_words = target_words.union(input_words)

In [8]:
# Tokenize
word_to_ix = {}
ix_to_word = {}

for num, i in enumerate(all_words):
    word_to_ix[i] = num
    ix_to_word[num] = i

In [9]:
word_to_ix['cell']

10141

In [10]:
ix_to_word[10141]

'cell'

In [14]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt.split()) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split()) for txt in target_texts])

In [17]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

MemoryError: 

In [16]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[word]] = 1.
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[word]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

NameError: name 'encoder_input_data' is not defined